In [1]:
#resnet

In [2]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

In [3]:
NFOLDS = 5

CACHE_PATH = '../input/modelfolder'

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(xx)

        dropout_rate = xx
        hidden_size = xx
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

In [5]:
device = torch.device("cuda:0")
resnet_model_list = []
for _fold in range(NFOLDS):
    torch.cuda.empty_cache()
    model = Model()
    model.to(device)
    model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    resnet_model_list.append(model)

In [6]:
resnet_model_list=resnet_model_list[-2:]

In [7]:
#keras

In [8]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
import random

import kerastuner as kt

In [9]:
keras_f_mean = np.load('../input/npyfolder/keras_f_mean_online.npy')

In [10]:
def create_autoencoder(input_dim,output_dim,noise=0.05):
    i = Input(input_dim)
    encoded = BatchNormalization()(i)
    encoded = GaussianNoise(noise)(encoded)
    encoded = Dense(64,activation='relu')(encoded)
    decoded = Dropout(0.2)(encoded)
    decoded = Dense(input_dim,name='decoded')(decoded)
    x = Dense(32,activation='relu')(decoded)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(output_dim,activation='sigmoid',name='label_output')(x)
    
    encoder = Model(inputs=i,outputs=decoded)
    autoencoder = Model(inputs=i,outputs=[decoded,x])
    
    autoencoder.compile(optimizer=Adam(0.001),loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, encoder

In [11]:
def create_model(hp,input_dim,output_dim,encoder):
    inputs = Input(input_dim)
    
    x = encoder(inputs)
    x = Concatenate()([x,inputs]) #use both raw and encoded features
    x = BatchNormalization()(x)
    x = Dropout(hp.Float('init_dropout',0.0,0.5))(x)
    
    for i in range(hp.Int('num_layers',1,3)):
        x = Dense(hp.Int(f'num_units_{i}',64,256))(x) #f
        x = BatchNormalization()(x)
        x = Lambda(tf.keras.activations.swish)(x)
        x = Dropout(hp.Float(f'dropout_{i}',0.0,0.5))(x)
    x = Dense(output_dim,activation='sigmoid')(x)
    model = Model(inputs=inputs,outputs=x)
    model.compile(optimizer=Adam(hp.Float('lr',0.00001,0.1,default=0.001)),loss=BinaryCrossentropy(label_smoothing=hp.Float('label_smoothing',0.0,0.1)),metrics=[tf.keras.metrics.AUC(name = 'auc')])
    return model

In [12]:
autoencoder, encoder = create_autoencoder(130,5,noise=0.1) 
encoder.load_weights('../input/modelfolder/encoder.hdf5')
encoder.trainable = False

In [13]:
model_fn = lambda hp: create_model(hp,130,5,encoder)

FOLDS = xx
SEED = xx

keras_models = []
hp = pd.read_pickle(f'../input/modelfolder/best_hp_{SEED}.pkl')
for f in range(FOLDS):
    model = model_fn(hp)
    model.load_weights(f'../input/modelfolder/model_{SEED}_{f}_finetune.hdf5')
    keras_models.append(model)

In [14]:
keras_model= keras_models[-1]
# keras_model= keras_models[-2]

In [15]:
#lgb
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [16]:
features = [col for col in feat_cols]
features.extend(['feature_stock_id_sum', 'feature_1_2_cross'])

In [17]:
import os, sys, gc, time, warnings, pickle, psutil, random
lgb_models = []
for i in range(5):
    model_path = '../input/modelfolder/lgb_model_'+str(i)+'.bin' 
    clf = pickle.load(open(model_path, 'rb'))
    lgb_models.append(clf)

In [18]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

In [19]:
test_df_columns = ['weight'] + [f'feature_{i}' for i in range(130)] + ['date']
index_features = [n for n, col in enumerate(test_df_columns) if col in feat_cols]

In [20]:
import gc
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].values[0]>0:
        #resnet
        resnet_x_tt = test_df.values[0][index_features].reshape(1, -1)
        if np.isnan(resnet_x_tt.sum()):
            resnet_x_tt = np.nan_to_num(resnet_x_tt) + np.isnan(resnet_x_tt) * f_mean
        cross_41_42_43 =resnet_x_tt[:, 41] +resnet_x_tt[:, 42] +resnet_x_tt[:, 43]
        cross_1_2 =resnet_x_tt[:, 1] / (resnet_x_tt[:, 2] + 1e-5)
        feature_inp = np.concatenate((
            resnet_x_tt,
            np.array(cross_41_42_43).reshape(resnet_x_tt.shape[0], 1),
            np.array(cross_1_2).reshape(resnet_x_tt.shape[0], 1),
            ), axis=1)
        resnet_pred = np.zeros((1, len(target_cols)))
        for model in resnet_model_list:
            resnet_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / 2
        resnet_pred = np.median(resnet_pred)
        #keras
        keras_x_tt = test_df.values[0][index_features].reshape(1, -1)
        if np.isnan(keras_x_tt.sum()):
            keras_x_tt = np.nan_to_num(keras_x_tt) + np.isnan(keras_x_tt) * keras_f_mean
        keras_pred = keras_model(keras_x_tt, training = False).numpy() 
        keras_pred =  np.median(keras_pred)
        #lgb_pred
        test_df['feature_stock_id_sum'] = test_df['feature_41'] + test_df['feature_42'] + test_df['feature_43']
        test_df['feature_1_2_cross'] = test_df['feature_1']/(test_df['feature_2']+1e-5)
        test_df.fillna(-9999,inplace=True)
        lgb_x_tt = test_df.loc[:, features].values
        lgb_pred = np.median([model.predict(lgb_x_tt) for model in lgb_models])
        # final_pred
        final_pred=0.4*resnet_pred +0.4*keras_pred+0.2*lgb_pred # try different weight
        pred_df.action = int(final_pred >= 0.5) #0.499,0.501
    else:
        pred_df['action'].values[0] = 0
    env.predict(pred_df)

15219it [05:01, 50.47it/s]
